In [1]:
import numpy as np
from qiskit import *
from qiskit.visualization import *

In [2]:
# PassPhrase=[]
n_qubits = 8
indx=0
PassPhrase=""
j=""
alice_random_string = np.random.randint(2, size = n_qubits)
for i in alice_random_string:
    j=str(i)
    PassPhrase = PassPhrase + j
print(PassPhrase)

00101100


In [3]:
from AESencryptfunc import * #import AESencryptfunc module to use functions created for this program
import math #import math module to use function such as ceiling

while(len(PassPhrase)!=16):
    print("Enter in the 16 character passphrase to encrypt your text file ")
    # PassPhrase=input()#takes in user input of char, eg. "Iwanttolearnkung"
    if(len(PassPhrase)<16):#check if less than 16 characters, if so add one space character until 16 chars
        while(len(PassPhrase)!=16):
            PassPhrase=PassPhrase+"\00"
    if(len(PassPhrase)>16):#check if bigger than 16 characters, if so then truncate it to be only 16 chars from [0:16]
        print("Your passphrase was larger than 16, truncating passphrase.")
        PassPhrase=PassPhrase[0:16]

#open plaintext.txt file to read and encrypt
sys.argv[1]="plaintext1.csv"
file=open(sys.argv[1], "r")
message=(file.read())
print("Inside your plaintext message is:\n%s\n" % message)
file.close()
sys.argv[2]="ciphertext.csv"
message=BitVector(textstring=message)
message=message.get_bitvector_in_hex()
replacementptr=0
while(replacementptr<len(message)):
    if(message[replacementptr:replacementptr+2]=='0a'):
        message=message[0:replacementptr]+'0d'+message[replacementptr:len(message)]
        replacementptr=replacementptr+4
    else:
        replacementptr=replacementptr+2

message=BitVector(hexstring=message)
message=message.get_bitvector_in_ascii()
#set up some parameters
start=0#set starting pointer for the part to encrypt of the plaintext
end=0#set ending pointer for the part to encrypt of the plaintex
length=len(message)#check the entire size of the message
loopmsg=0.00#create a decimal value
loopmsg=math.ceil(length/16)+1#use formula to figure how long the message is and how many 16 character segmentss must be encrypted
outputhex=""#setup output message in hex

#need to setup roundkeys here
PassPhrase=BitVector(textstring=PassPhrase)
roundkey1=findroundkey(PassPhrase.get_bitvector_in_hex(),1)
roundkey2=findroundkey(roundkey1,2)
roundkey3=findroundkey(roundkey2,3)
roundkey4=findroundkey(roundkey3,4)
roundkey5=findroundkey(roundkey4,5)
roundkey6=findroundkey(roundkey5,6)
roundkey7=findroundkey(roundkey6,7)
roundkey8=findroundkey(roundkey7,8)
roundkey9=findroundkey(roundkey8,9)
roundkey10=findroundkey(roundkey9,10)
roundkeys=[roundkey1,roundkey2,roundkey3,roundkey4,roundkey5,roundkey6,roundkey7,roundkey8,roundkey9,roundkey10]

#set up FILEOUT to write
FILEOUT = open(sys.argv[2], 'w')

# set up the segement message loop parameters
for y in range(1, loopmsg): # loop to encrypt all segments of the message
    if(end+16<length): #if the end pointer is less than the size of the message, then set the segment to be 16 characters
        plaintextseg = message[start:end + 16]
    else: #or else if the end pointer is equal to or greator than the size of the message
        plaintextseg = message[start:length]
        for z in range(0,((end+16)-length),1): #run a while loop to pad the message segement to become 16 characters, if it is 16 already the loop will not run
            plaintextseg = plaintextseg+"\00"

    #add round key zero/ find round key one
    bv1 = BitVector(textstring=plaintextseg)
    bv2 = PassPhrase
    resultbv=bv1^bv2
    myhexstring = resultbv.get_bitvector_in_hex()

    for x in range(1, 10):  # loop through 9 rounds
        # sub byte
        myhexstring = resultbv.get_bitvector_in_hex()
        temp1=subbyte(myhexstring)

        # shift rows
        temp2=shiftrow(temp1)

        # mix column
        bv3 = BitVector(hexstring=temp2)
        newbvashex=mixcolumn(bv3)
        newbv=BitVector(hexstring=newbvashex)

        #add roundkey for current round
        bv1 = BitVector(bitlist=newbv)
        bv2 = BitVector(hexstring=roundkeys[x-1])
        resultbv = bv1 ^ bv2
        myhexresult = resultbv.get_bitvector_in_hex()

    #start round 10
    # sub byte round 10
    myhexstring = resultbv.get_bitvector_in_hex()
    temp1=subbyte(myhexstring)

    # shift rows round 10
    temp2=shiftrow(temp1)

    # add round key round 10
    newbv = BitVector(hexstring=temp2)
    bv1 = BitVector(bitlist=newbv)
    bv2 = BitVector(hexstring=roundkeys[9])
    resultbv = bv1 ^ bv2
    myhexstring = resultbv.get_bitvector_in_hex()

    #set encrypted hex segement of message to output string
    outputhextemp = resultbv.get_hex_string_from_bitvector()
    FILEOUT.write(outputhextemp)
    start = start + 16 #increment start pointer
    end = end + 16 #increment end pointer

# encrypted output hex string to specified cipherhex file
FILEOUT.close()

file2=open(sys.argv[2], "r")
print("The output hex value for the entire message is:\n%s\n" % file2.read())
file2.close()

Enter in the 16 character passphrase to encrypt your text file 
Inside your plaintext message is:
name,pokedex id,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
bulbasaur,1,7,69,grass,poison,45,49,49,65,65,45
ivysaur,2,10,130,grass,poison,60,62,63,80,80,60
venusaur,3,20,1000,grass,poison,80,82,83,100,100,80
charmander,4,6,85,fire,None,39,52,43,60,50,65

The output hex value for the entire message is:
aeabf54b137d41857f493d70b070613b4d3ebba1e5aae154577abe2d19b6ea71ca6b5c49525a0a0aa849026e10c4d85a38743d42dfa3beb931bdc2ac7e8062048bffd5443111fe21e682671b37333da6c426b537f6123f23ecb85958a221f99e46a042612b6dd53ffdf61165f24be209394544910010e9698bb8f8b7fbab53136e6d25f66c0aa556732afb294aef7bfaae22019a1a616e27eec1f2d31294e7db768fb6f1c4ffeb1bd78930e6d4884d39f5905db6ffcdf8c9e2792a2b6422fc32284748aa69787815fa8aa22596c1b8e2dd68032c26c1c484642960a82d93049493aba4dcaa5929a10475246f292a20c72697d805eed75065c348dff5b434bc7996f29c1efaa2cf0a30c8282f931d87f91abf94a9ef10cfaa2b676db51a5

# B92 Protocol

## Protocol Steps
1. Alice generates a random bit string and creates a corresponding qubit string that is mapped from the bit string as follows: 0 -> |z+>, 1 -> |x+>. Alice then sends these qubits to Bob over a public channel
2. Bob generates a random bit string and measures Alice's qubits with the following mapping from his bit string: 0 -> {|x+>, |x->}, 1: -> {|z+>, |z->}
3. Bob sends his measured bits back to Alice over a public channel.
4. Alice and Bob both remoove all the bits in their strings corresponding to 0s in the string bob just sent

### Import packages

In [4]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd

### Define all the meta parameters we will need for the protocol
- n: the number of bits that Alice and Bob will randomly generate in the protocol
- test: boolean that will standardize the pseudo-random fucntions in the protocol and print the steps of the protocol along the way

In [5]:
n = n_qubits
test = True
rand_seed = False

In [6]:
if rand_seed:
    np.random.seed(seed=0)

### Create the qubits that Alice will send across the public channel

In [7]:
def alice_encode(input_arr):
    output = []
    for bit in input_arr:
        qc = QuantumCircuit(1,1)
        
        if bit == 1:
            qc.h(0)
        qc.barrier()
        output.append(qc)
    return output

In [8]:
alice_bits = alice_random_string
alice_qubits = alice_encode(alice_bits)

alice_bits_str = ''.join(map(str, alice_bits))

if test:
    
    print('Alice\'s randomly generated bits:')
    print(alice_bits)
    
    print('Alice qubits:')
    for i in alice_qubits:
        print(i)


Alice's randomly generated bits:
[0 0 1 0 1 1 0 0]
Alice qubits:
      ░ 
  q: ─░─
      ░ 
c: 1/═══
        
      ░ 
  q: ─░─
      ░ 
c: 1/═══
        
     ┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             
      ░ 
  q: ─░─
      ░ 
c: 1/═══
        
     ┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             
     ┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════
             
      ░ 
  q: ─░─
      ░ 
c: 1/═══
        
      ░ 
  q: ─░─
      ░ 
c: 1/═══
        


### Bob measures the qubits and sends a bit string back to Alice across the public channel
- We define a function that take a list of qubits and a list of bits. We loop through the lists in parallel and if the corresponding bit is a 0, we measure the qubit in the x-basis and in the z-basis otherwise. We return all of the outputs in a list

In [9]:
def bob_decode(alice_qubits, bob_bits):
    output = []
    for alice_qubit, bob_bit in zip(alice_qubits, bob_bits):
        
        # Measure in the x-basis
        if bob_bit == 0:
            alice_qubit.h(0)
            alice_qubit.measure(0, 0)
        
        # Measure in the z-basis
        else:
            alice_qubit.measure(0, 0)
            
        # Simulate the measurement (taken directly from the qiskit example)
        backend = Aer.get_backend('qasm_simulator')
        qobj = assemble(alice_qubit, shots=1, memory=True)
        execution = execute(experiments = alice_qubit.reverse_bits(), backend = backend, shots = 10)    # Shots = 1
        results = execution.result()
        counts = results.get_counts()
        
        output.append(counts)
    return output    

In [10]:
bob_bits = randint(2, size=n)
bob_yield = bob_decode(alice_qubits, bob_bits)

if test:
    print('Bob\'s randomly generated bits:')
    print(bob_bits)
    
    print('Bob\'s measurement yeild:')
    print(bob_yield)


Bob's randomly generated bits:
[0 1 1 0 0 1 0 1]
Bob's measurement yeild:
[{'1': 5, '0': 5}, {'0': 10}, {'0': 5, '1': 5}, {'0': 6, '1': 4}, {'0': 10}, {'0': 6, '1': 4}, {'1': 5, '0': 5}, {'0': 10}]


### Create the shared keys
- We define a function that takes both Alice and Bobs original strings and removes all of the bits that correspond to a 0 in the bits Bob measured. We return a tuple of stirngs that are a concatenation of these bits

In [11]:
ans=""
arr=[]
input_key=input("Enter the key")
for i in input_key:
    arr.append(i)
print(arr)
for i in range(0,len(arr)):
    ans=ans+arr[i]       
print(ans)

['0', '0', '1', '0', '1', '1', '0', '0']
00101100


In [12]:
def remove_zeros(alice_bits, bob_bits, bob_yield):
    bob_key = ''
    alice_key = ''
    
    for a_bit, b_bit, b_yield in zip(alice_bits, bob_bits, bob_yield):
        if b_yield == 1:
            bob_key += str(b_bit)
            alice_key += str(a_bit)

    return alice_key, bob_key

In [13]:
# alice_key, bob_key = remove_zeros(alice_bits, bob_bits, bob_yield)
alice_key=alice_bits_str
bob_key =input_key
if test:
    print('Key strings')
    print(f'Alice\'s key: {alice_key}')
    print(f'Bob\'s key: {bob_key}')

Key strings
Alice's key: 00101100
Bob's key: 00101100


### Testing Accuracy
- We can test if the protocol worked and Alice and Bob have the same keys

In [14]:
if bob_key == alice_key:
    print('The protocol worked and Alice and Bob have the same keys!')
else:
    print('Something went wrong! Alice and Bob do not have the same keys!')

The protocol worked and Alice and Bob have the same keys!


In [15]:
from AESdecryptfunc import * 
import math 
import io

sys.argv[1]="ciphertext.csv"
sys.argv[2]="plaintext2.csv"
ans=bob_key
while(len(PassPhrase)!=16):
    print("Enter in the 16 character passphrase to decrypt your text file %s" %sys.argv[1])
    PassPhrase=ans
    if(len(PassPhrase)<16):
        while(len(PassPhrase)!=16):
            PassPhrase=PassPhrase+"\00"
    if(len(PassPhrase)>16):#check if bigger than 16 characters, if so then truncate it to be only 16 chars from [0:16]
        print("Your passphrase was larger than 16, truncating passphrase.")
        PassPhrase=PassPhrase[0:16]

#open ciphertext.txt file to read and decrypt
file=open(sys.argv[1], "r")
message=(file.read())
print("Inside your ciphertext message is:\n%s\n" % message)
file.close()

#set up some parameters
start=0#set starting pointer for the part to decrypt of the ciphertext
end=32#set ending pointer for the part to decrypt of the plaintex
length=len(message)#check the entire size of the message
loopmsg=0.00#create a decimal value
loopmsg=math.ceil(length/32)+1#use formula to figure how long the message is and how many 16 character segments must be decrypted
outputhex=""#setup output message segment in hex
asciioutput=""#setup compilation of output message in ascii

#need to setup roundkeys here
PassPhrase=BitVector(textstring=PassPhrase)
roundkey1=findroundkey(PassPhrase.get_bitvector_in_hex(),1)
roundkey2=findroundkey(roundkey1,2)
roundkey3=findroundkey(roundkey2,3)
roundkey4=findroundkey(roundkey3,4)
roundkey5=findroundkey(roundkey4,5)
roundkey6=findroundkey(roundkey5,6)
roundkey7=findroundkey(roundkey6,7)
roundkey8=findroundkey(roundkey7,8)
roundkey9=findroundkey(roundkey8,9)
roundkey10=findroundkey(roundkey9,10)
roundkeys=[roundkey1,roundkey2,roundkey3,roundkey4,roundkey5,roundkey6,roundkey7,roundkey8,roundkey9,roundkey10]

FILEOUT = io.open(sys.argv[2], 'w', encoding='utf-8')

# set up the segement message loop parameters
for y in range(1, loopmsg): # loop to encrypt all segments of the message
    plaintextseg = message[start:end]

    # add round key
    bv1 = BitVector(hexstring=plaintextseg)
    bv2 = BitVector(hexstring=roundkeys[9])
    resultbv = bv1 ^ bv2
    myhexstring = resultbv.get_bitvector_in_hex()

    #inverse shift row
    myhexstring=invshiftrow(myhexstring)

    #inverse subbyte
    myhexstring=invsubbyte(myhexstring)

    for x in range(8, -1, -1):
        # add roundkey for current round
        bv1 = BitVector(hexstring=myhexstring)
        bv2 = BitVector(hexstring=roundkeys[x])
        resultbv = bv1 ^ bv2
        myhexstring = resultbv.get_bitvector_in_hex()

        # mix column
        bv3 = BitVector(hexstring=myhexstring)
        myhexstring=invmixcolumn(bv3)

        # shift rows
        myhexstring = invshiftrow(myhexstring)

        # sub byte
        myhexstring = invsubbyte(myhexstring)

    #add initial round key
    bv1 = BitVector(hexstring=myhexstring)
    bv2 = PassPhrase
    resultbv = bv1 ^ bv2
    myhexstring = resultbv.get_bitvector_in_hex()

    start = start + 32 #increment start pointer
    end = end + 32 #increment end pointer

    replacementptr = 0
    while (replacementptr < len(myhexstring)):
        if (myhexstring[replacementptr:replacementptr + 2] == '0d'):
            myhexstring = myhexstring[0:replacementptr] + myhexstring[replacementptr+2:len(myhexstring)]
        else:
            replacementptr = replacementptr + 2

    outputhex = BitVector(hexstring=myhexstring)
    asciioutput = outputhex.get_bitvector_in_ascii()
    asciioutput=asciioutput.replace('\x00','')
    FILEOUT.write(asciioutput)

FILEOUT.close()

file2=io.open(sys.argv[2], "r", encoding='utf-8')
print("The decrypted message for the entire ciphertext is:\n%s\n" % file2.read())
file2.close()

Enter in the 16 character passphrase to decrypt your text file ciphertext.csv
Inside your ciphertext message is:
aeabf54b137d41857f493d70b070613b4d3ebba1e5aae154577abe2d19b6ea71ca6b5c49525a0a0aa849026e10c4d85a38743d42dfa3beb931bdc2ac7e8062048bffd5443111fe21e682671b37333da6c426b537f6123f23ecb85958a221f99e46a042612b6dd53ffdf61165f24be209394544910010e9698bb8f8b7fbab53136e6d25f66c0aa556732afb294aef7bfaae22019a1a616e27eec1f2d31294e7db768fb6f1c4ffeb1bd78930e6d4884d39f5905db6ffcdf8c9e2792a2b6422fc32284748aa69787815fa8aa22596c1b8e2dd68032c26c1c484642960a82d93049493aba4dcaa5929a10475246f292a20c72697d805eed75065c348dff5b434bc7996f29c1efaa2cf0a30c8282f931d87f91abf94a9ef10cfaa2b676db51a58deaa

The decrypted message for the entire ciphertext is:
name,pokedex id,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
bulbasaur,1,7,69,grass,poison,45,49,49,65,65,45
ivysaur,2,10,130,grass,poison,60,62,63,80,80,60
venusaur,3,20,1000,grass,poison,80,82,83,100,100,80
charmander,4,6,85,fir